# Analyse de la qualité des données du fichier produits 
Cet extract CSV provient du fournisseur

## Résumé des anomalies observées
- 8% des produits n'ont pas de référence (null)
--> problème de complétude : l'incrément ne s'est pas fait
- 10% des produits ont un prix négatif 
<br>--> erreur de saisie (problème saisie manuelle)
- 100% des produits ont plus de chiffres après la virgule
  <br> --> erreur de cohérence (problème de conversion d'une devise étrangère)
- 100% des produits ont une description commençant par 'Produit' et réplique ce champs
<br> + problème de cohérence : le dernier mot ne correspond pas à une description
<br>--> erreur de contamination/redondance (problème de mapping)
<br><br>
Pas de doublon (aucune ligne dupliquée)

## Proposition d'une correction
- les références manquantes peuvent être retrouvées par interpolation (n-1 / n+1)
<br> --> 100% de correction validée

In [2]:
import pandas as pd
import numpy as np

In [3]:
df_products = pd.read_csv('../data/raw/Catalogue_Produits.csv')

In [9]:
print("AUDIT DATAFRAME MARKETPLACE")

display(df_products.head())
display(df_products.info())

print("\n---Taux de valeurs manquantes (%)---")
missing_percentage = df_products.isnull().sum() / len(df_products) * 100
print(missing_percentage[missing_percentage > 0].sort_values(ascending=False))

print("\n---Vérification des doublons---")
print("Nombre total de doublons :", df_products.duplicated().sum())

print("\n--- Vérification des doublons sur ref_produits (autres que null) ---")
df_non_null = df_products[df_products['ref_produit'].notna()]
nb_doublons = df_non_null['ref_produit'].duplicated().sum()
print(f"Nombre de références produits en doublon : {nb_doublons}")
if nb_doublons > 0:
    # Afficher les références en doublon
    refs_dupliquees = df_products[df_products['ref_produit'].duplicated(keep=False)]
    print(f"\nNombre total de lignes concernées : {len(refs_dupliquees)}")

    # Afficher quelques exemples de lignes dupliquées
    print("\nExemples de lignes avec références dupliquées :")
    display(refs_dupliquees.sort_values('ref_produit').head(10))

print("\n---Vérification des prix négatifs---")
print("Nombre total de prix négatifs :",(df_products['prix_fournisseur']<0).sum())

print("\n--- Analyse de la précision des prix ---")
prix_anormaux = df_products['prix_fournisseur'].apply(lambda x: len(str(x).split('.')[-1]) > 2 if '.' in str(x) else False)
print(f"Nombre de prix avec >2 décimales : {prix_anormaux.sum()}")
print(f"{prix_anormaux.sum()/len(df_products) * 100}% des produits ont plus de 2 décimales")
print("Il doit donc y avoir un problème de conversion des prix.")

print("\n--- Analyse du mot 'produit' dans la description ---")
# Vérifier si la description commence exactement par "Produit " suivi du nom du produit
def verifie_format_produit(row):
    if pd.isna(row['description']) or pd.isna(row['produit']):
        return False

    format_attendu = f"Produit {row['produit']}"
    return row['description'].startswith(format_attendu)

mask_format_produit = df_products.apply(verifie_format_produit, axis=1)

nb_avec_format = mask_format_produit.sum()

print(f"Nombre de lignes où description = 'Produit ' + produit : {nb_avec_format}")
# print(f"Pourcentage : {(nb_avec_format / len(df_products)) * 100:.2f}%")
print(f"{nb_avec_format/len(df_products) * 100:.2f}% des descriptions commencent par 'Produit' et répliquent ce champs")
print("Il doit donc y avoir un problème dans la configuration de l'export côté fournisseur.")

AUDIT DATAFRAME MARKETPLACE


,produit,ref_produit,description,prix_fournisseur,date_mise_a_jour
0,Écran 24 pouces,REF-1001,Produit Écran 24 pouces figure,66.556477,2025-07-26
1,Webcam HD,REF-1002,Produit Webcam HD aussi,121.868920,2024-09-23
2,Casque Bluetooth,REF-1003,Produit Casque Bluetooth choisir,59.158904,2023-02-22
3,Enceinte Bluetooth,NaN,Produit Enceinte Bluetooth chercher,-21.284130,2022-12-03
4,Chargeur rapide,REF-1005,Produit Chargeur rapide sien,97.507855,2023-02-08


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   produit           100 non-null    object 
 1   ref_produit       92 non-null     object 
 2   description       100 non-null    object 
 3   prix_fournisseur  100 non-null    float64
 4   date_mise_a_jour  100 non-null    object 
dtypes: float64(1), object(4)
memory usage: 4.0+ KB


None


---Taux de valeurs manquantes (%)---
ref_produit    8.0
dtype: float64

---Vérification des doublons---
Nombre total de doublons : 0

--- Vérification des doublons sur ref_produits (autres que null) ---
Nombre de références produits en doublon : 0

---Vérification des prix négatifs---
Nombre total de prix négatifs : 10

--- Analyse de la précision des prix ---
Nombre de prix avec >2 décimales : 100
100.0% des produits ont plus de 2 décimales
Il doit donc y avoir un problème de conversion des prix.

--- Analyse du mot 'produit' dans la description ---
Nombre de lignes où description = 'Produit ' + produit : 100
100.00% des descriptions commencent par 'Produit' et répliquent ce champs
Il doit donc y avoir un problème dans la configuration de l'export côté fournisseur.


## Correction des incréments manquants pour ref_produit

In [12]:
print("\n--- CORRECTION DES RÉFÉRENCES MANQUANTES PAR ITERATION etINTERPOLATION ---")

# copie du dataframe original
df_products_corrected = df_products.copy()

# Fonction pour extraire le numéro d'une référence
def extraire_numero(ref):
    """Extrait le numéro d'une référence type REF-1006"""
    if pd.isna(ref):
        return None
    try:
        return int(str(ref).split('-')[-1])
    except:
        return None

# Fonction pour construire une référence
def construire_ref(numero):
    """Construit une référence type REF-1006"""
    return f"REF-{numero}"

# Itérer sur chaque ligne
nb_corriges = 0

for i in range(len(df_products_corrected)):
    # Vérifier si la référence actuelle est manquante
    if pd.isna(df_products_corrected.loc[i, 'ref_produit']):

        # Récupérer le numéro de la ligne précédente (n-1)
        if i > 0:
            num_precedent = extraire_numero(df_products_corrected.loc[i-1, 'ref_produit'])
        else:
            num_precedent = None

        # Récupérer le numéro de la ligne suivante (n+1)
        if i < len(df_products_corrected) - 1:
            num_suivant = extraire_numero(df_products_corrected.loc[i+1, 'ref_produit'])
        else:
            num_suivant = None

        # Vérifier si l'incrément est de +2
        if num_precedent is not None and num_suivant is not None:
            if num_suivant == num_precedent + 2:
                # Calculer la valeur manquante (n-1 + 1)
                num_manquant = num_precedent + 1

                # Inscrire la référence corrigée
                df_products_corrected.loc[i, 'ref_produit'] = construire_ref(num_manquant)
                nb_corriges += 1

print(f"✅ {nb_corriges} références corrigées")
print(f"Références encore nulles : {df_products_corrected['ref_produit'].isna().sum()}")



--- CORRECTION DES RÉFÉRENCES MANQUANTES PAR ITERATION etINTERPOLATION ---
✅ 8 références corrigées
Références encore nulles : 0


In [13]:
print("\n--- SAUVEGARDE DU FICHIER CORRIGÉ ---")

# Sauvegarder dans le dossier processed
df_products_corrected.to_csv('../data/processed/Catalogue_Produits_corrected.csv', index=False)


--- SAUVEGARDE DU FICHIER CORRIGÉ ---
